In [1]:
import keras
import numpy as np
import json

text = open(r'The_Lord_of_the_Rings','r',encoding="utf-8_sig")
text = text.read().lower()
print('Corpus length:', len(text))

C:\Users\apple\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Corpus length: 1010402


In [2]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 336781
Unique characters: 56
Vectorization...


In [3]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

model.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               94720     
_________________________________________________________________
dense_1 (Dense)              (None, 56)                7224      
Total params: 101,944
Trainable params: 101,944
Non-trainable params: 0
_________________________________________________________________


In [4]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [6]:
import random
import sys

for epoch in range(1, 6):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    
    # this time, we choose only 1.0 temperature to make the training time short
    for temperature in [1.0]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
336781/336781 [==============================] - 533s 2ms/step - loss: 1.7753
--- Generating with seed: "k a deep draught of the air, and felt that a skip and a few "
------ temperature: 1.0
k a deep draught of the air, and felt that a skip and a few betten lied, from the ecoing, in the punse-xeep you wanf hey in accounllibut to gimp. an tretfur, and near 
toom, under a murse. foundains to see of barger, and the feat striver an-somes of the harning heem, shall fresters of into tr all coupshert between of bark brough, the same. a breapule. and egrem be in from boriel. but then thet dilk him. a growsed findless was trading acceaks and e,ply. a b
epoch 2
Epoch 1/1
336781/336781 [==============================] - 510s 2ms/step - loss: 1.5019
--- Generating with seed: " 
the hobbits lay down on their blankets with their feet tow"
------ temperature: 1.0
 
the hobbits lay down on their blankets with their feet tower. 
 folly year 
thevica for broken light, that we shall dee